<a href="https://colab.research.google.com/github/eddiechu888/idioms/blob/main/idioms_llama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import seaborn as sns
from getpass import getpass

# 1. Setup
hf_token = getpass('Enter your Hugging Face token: ')

if hf_token is None:
    raise ValueError("HUGGINGFACE_TOKEN is not set.")

model_name = 'meta-llama/Llama-3.2-1B'

# Check for CUDA and MPS availability and use the best available option
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Initialize tokenizer and model with the HF token for authentication
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)

# Set pad_token to eos_token if pad_token is not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Pad token not found. Setting pad_token to eos_token: {tokenizer.pad_token}")
else:
    print(f"Using existing pad_token: {tokenizer.pad_token}")

model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token).to(device)
model.eval()  # Set model to evaluation mode

# 2. Define the Single Data Example
single_example = {
    "idiomatic": "She thought of good ways to break the ice at the company retreat.",
    "non_idiomatic": "She thought of good ways to get people talking at the company retreat.",
    "literal": "She thought of good ways to break the ice to make a cold drink."
}

# 3. Define Hook for Activations
activations = defaultdict(list)

def get_activation(name):
    def hook(model, input, output):
        # If output is a tuple, extract the first element (hidden states)
        if isinstance(output, tuple):
            output = output[0]

        # If output is a list, extract the first element
        if isinstance(output, list):
            output = output[0]

        # Check if the output is a Tensor
        if isinstance(output, torch.Tensor):
            print(f"Hook '{name}': Capturing Tensor with shape {output.shape}")
            activations[name].append(output.detach().cpu())
        else:
            print(f"Hook '{name}': Output is of type {type(output)}, skipping.")
    return hook

# Register hooks on desired layers
# It's crucial to inspect the model architecture to correctly access the transformer layers.
# For LLaMA models, the transformer layers are typically under model.model.layers
# Adjust the attribute path if necessary.

for i, layer in enumerate(model.model.layers):
    layer_name = f'layer_{i}'
    layer.register_forward_hook(get_activation(layer_name))

print("Hooks registered on model layers.")

# 4. Function to Process a Sentence and Capture Activations
def process_sentence(sentence, max_length=20):
    encoded_input = tokenizer(
        sentence,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=max_length
    ).to(device)
    with torch.no_grad():
        model(**encoded_input)
    # After forward pass, activations are populated via hooks
    return

# 5. Process Each Sentence in the Triplet
conditions = ['idiomatic', 'non_idiomatic', 'literal']
condition_activations = {condition: defaultdict(list) for condition in conditions}

for condition in conditions:
    sentence = single_example[condition]
    print(f"\nProcessing '{condition}' sentence:")
    print(f"Sentence: {sentence}")

    # Clear previous activations
    activations.clear()

    # Process the sentence
    process_sentence(sentence)

    # Store activations
    for layer, act in activations.items():
        condition_activations[condition][layer].extend(act)

    print(f"Activations captured for '{condition}' condition.")

# 6. Analyze and Compare Activations
# For simplicity, we'll compute the average activation for each layer and visualize differences.

def compute_average_activation(activations_dict):
    average_activation = {}
    for layer, acts in activations_dict.items():
        # Stack all activations for the layer (should be only one in this case)
        stacked = torch.cat(acts, dim=0)  # Shape: [batch_size, ...]
        # Compute the mean across the batch dimension
        average = stacked.mean(dim=0)
        average_activation[layer] = average
    return average_activation

avg_idiomatic = compute_average_activation(condition_activations['idiomatic'])
avg_non_idiomatic = compute_average_activation(condition_activations['non_idiomatic'])
avg_literal = compute_average_activation(condition_activations['literal'])

# Example: Compare average activations of a specific layer
layer_to_compare = 'layer_0'  # Change as needed

idiom_avg = avg_idiomatic[layer_to_compare]
non_idiom_avg = avg_non_idiomatic[layer_to_compare]
literal_avg = avg_literal[layer_to_compare]

# Compute differences
difference_idiom_non_idiom = idiom_avg - non_idiom_avg
difference_idiom_literal = idiom_avg - literal_avg

print(f"\nAverage activation difference for {layer_to_compare} (Idiomatic - Non-Idiomatic): {difference_idiom_non_idiom}")
print(f"Average activation difference for {layer_to_compare} (Idiomatic - Literal): {difference_idiom_literal}")

# 7. Visualize Activation Differences
# We'll plot the activation differences as heatmaps.

def plot_activation_difference(difference_tensor, layer_name, condition_pair):
    difference_np = difference_tensor.numpy()

    plt.figure(figsize=(12, 6))
    sns.heatmap(difference_np, cmap='coolwarm')
    plt.title(f'Activation Differences in {layer_name} ({condition_pair})')
    plt.xlabel('Neuron Index')
    plt.ylabel('Hidden Units' if len(difference_np.shape) > 1 else 'Dimension')
    plt.show()

# Plotting the differences
plot_activation_difference(difference_idiom_non_idiom, layer_to_compare, 'Idiomatic - Non-Idiomatic')
plot_activation_difference(difference_idiom_literal, layer_to_compare, 'Idiomatic - Literal')

Enter your Hugging Face token: ··········
Using device: cuda
Hooks registered on model layers.

Processing 'idiomatic' sentence:
Sentence: She thought of good ways to break the ice at the company retreat.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.